In [7]:
import pickle
from checklist.editor import Editor
from checklist.pred_wrapper import PredictorWrapper
from checklist.test_suite import TestSuite
from checklist.test_types import MFT
import numpy as np
import pandas as pd

In [5]:
def predict_proba(inputs):
    p = np.array([model.predict_proba([x]) for x in inputs]).reshape(-1, 2)
    
    return p
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

# EDA
Before we can do any kind of checklisting, we need to know what kind of nouns we have in the dataset for our vocabulary, as well as which sentences are most common and get a general understanding of the dataset

## Beauty

In [8]:
df_beauty = pd.read_csv("data\\beauty_reviews(subset).csv")
df_beauty

,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,0,US,25245446,R1VTEU1P0T413X,B004NHVTZQ,417544650,Royale E038 Classic 1.5 Inch Black Flat Iron,Beauty,4,0,0,N,N,Great flat iron.,Great flat iron unfortunately I bought this at...,2014-12-10,positive
1,1,US,41673718,R1FQJ6NOXV3ESI,B00GBYNAJM,382525561,Sexy Red Lingerie Sequin Heart Tassel Breast B...,Beauty,5,0,0,N,Y,Super cute! I received a different pair that h...,Super cute! I received a different pair that h...,2015-04-08,positive
2,2,US,41448631,RPHARNZ4IXL8D,B006GU1B5Q,283496396,Iso Beauty Ceramic Pro Flat Iron Straightener ...,Beauty,5,2,2,N,Y,Great product,Good quality straightener. It gets super hot a...,2013-11-09,positive
3,3,US,45153186,R1NMN4RANL21D2,B00J6SW13Q,332394087,SEPHORA COLLECTION Together in Pink Brush Set ...,Beauty,1,0,2,N,Y,One Star,"Actually，I have got a book,wasn't this Pink Br...",2014-12-14,negative
4,4,US,44124985,RRSUKHUHPKVS9,B000ELYIVM,413825976,Dental Hygiene Preferred Complete Day/Night M...,Beauty,5,0,1,N,Y,Good cheap mouthguard(s),These are good and durable. Comes with 2 to ge...,2010-12-30,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081376,1081376,US,15443038,R2TQLKQ5TYSBRX,B00158O1EM,897391812,Norelco 7735X Cool Skin Additive In/Out of Sho...,Beauty,5,3,3,N,N,Cool Face Saver,I was hesitant that this would work well. As a...,2009-01-25,positive
1081377,1081377,US,20640811,RETGMD5GBUJ6W,B0028RBNYK,107073507,Century Shampoo Scalp Massage Brush #100 * Blu...,Beauty,5,0,0,N,Y,Great product!,"These are fantastic. They work well, feel real...",2011-06-15,positive
1081378,1081378,US,18061495,R34N7UHVVTUBKO,B002FGKPWK,272867906,The Body Shop Exfoliating Skin Towel,Beauty,2,1,1,N,Y,Doesn't last long,Started coming apart at the seams after a coup...,2014-02-12,negative
1081379,1081379,US,8616203,R1ZTY1W7AMNBP8,B003983I1S,555191963,Gillette Fusion5 ProGlide Men's Razor Blades R...,Beauty,5,0,0,N,N,Closest Shave,I recently was able to try out the new ProGlid...,2013-06-28,positive


In [10]:
df_beauty["review_body"].value_counts()

good                                                                                                                                                                                                                                                                                                                                                                                                     1362
Good                                                                                                                                                                                                                                                                                                                                                                                                     1281
Great                                                                                                                                                                                                       

# Checklisting

## Loading models

In [3]:
with open("models\\baseline_model_beauty.pickle", "rb") as file:
    model_beauty = pickle.load(file)

In [4]:
with open("models\\baseline_model_sports.pickle", "rb") as file:
    model_sports = pickle.load(file)

## Beauty

## Capability: Vocabulary

In [ ]:
editor = Editor()
suite = TestSuite()

In [ ]:
nouns = ['product','wig','lipstick','cream']
editor.add_lexicon('nouns', nouns)
pos_adj = ['amazing', 'incredible', 'extraordinary', 'excellent', 'awesome', 'nice']
neg_adj = ['terrible', 'eerie', 'odd', 'awful', 'ugly', 'strange']
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )

# Hardcases

## Fairness
For fairness we want to see if our participents annotate differently based on the gender of the reviewer, or any minority background the reviewer might have. 

They are also selected based on if the product is targeted at a specific gender or if the creator of the product might be a specific gender. These reviews are selected from the dataset, where these details are overtly expressed, with a few exceptions. 

These exceptions are when a product line's name is mentioned and that product line is targeted at a specific gender or if a creator of a product is a well known person and we know their gender or minority background based on their name being present in the review. As an example, if a review states "This Nicki Minaj album is great!", we know that this album is made by a woman of colour.